# Clustering Ground Truth

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# get .txt of all files from download
# ! cd KAR_S1 
# !find -mindepth 3 -printf "%P\n" > ../KGA_S1.csv/KGA_inventory.txt

In [3]:
folder = 'ENO' # 'KGA' 'KAR' 'KRU' 'CDB'
path = './' + folder + '_S1.csv/' + folder + '_S1_report_lila.csv'
inventory_path = './' + folder + '_S1.csv/' + folder + '_S1_report_lila_image_inventory.csv'

true_inventory = './' + folder + '_S1.csv/' + folder + '_inventory.txt'

In [4]:
# downloaded set
df_inv = pd.read_csv(true_inventory, delimiter = "\t", names=['image_path_rel'])
df_inv['image_path_rel'] = folder + '_S1/' + df_inv['image_path_rel']
df_inv

,image_path_rel
0,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0017.JPG
1,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0018.JPG
2,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0019.JPG
3,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0020.JPG
4,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0021.JPG
...,...
28539,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0141.JPG
28540,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0142.JPG
28541,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0143.JPG
28542,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0144.JPG


In [5]:
# metadata: sequence id <-> images
df1 = pd.read_csv(inventory_path)
df1 = df1.join(df_inv.set_index('image_path_rel'), on='image_path_rel', how='inner')[['capture_id', 'image_path_rel']]
print(len(df1['image_path_rel'].unique()))
df1

28544


,capture_id,image_path_rel
15,ENO_S1#B02#1#7,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0017.JPG
16,ENO_S1#B02#1#7,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0018.JPG
17,ENO_S1#B02#1#7,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0019.JPG
18,ENO_S1#B02#1#8,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0020.JPG
19,ENO_S1#B02#1#8,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0021.JPG
...,...,...
29409,ENO_S1#F06#1#48,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0141.JPG
29410,ENO_S1#F06#1#48,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0142.JPG
29411,ENO_S1#F06#1#49,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0143.JPG
29412,ENO_S1#F06#1#49,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0144.JPG


In [6]:
# metadata: id contents
df2_cols = ['capture_id', 'site', 'capture_time_local', 'question__species'] #'capture_date_local', 
df2 = pd.read_csv(path)
df2 = df2[df2_cols]
df2

,capture_id,site,capture_time_local,question__species
0,ENO_S1#B02#1#2,B02,09:38:46,human
1,ENO_S1#B02#1#3,B02,09:39:46,human
2,ENO_S1#B02#1#4,B02,09:42:03,human
3,ENO_S1#B02#1#5,B02,09:43:46,human
4,ENO_S1#B02#1#6,B02,09:44:47,human
...,...,...,...,...
13694,ENO_S1#F06#1#45,F06,14:16:46,blank
13695,ENO_S1#F06#1#46,F06,14:20:09,domesticanimal
13696,ENO_S1#F06#1#47,F06,11:51:54,blank
13697,ENO_S1#F06#1#48,F06,11:52:57,blank


In [7]:
# inner join: get info per image in download set
df_full = df2.copy().join(df1.set_index('capture_id'), on='capture_id', how='inner').drop_duplicates('image_path_rel')

sites = df_full['site'].unique()

dataset_count = {}
for s in sites: 
    num = np.int(df_full.loc[df_full['site'] == s]['capture_id'].count())
    
    dataset_count[str(s)] = num

print(json.dumps(dataset_count, indent=2))

total = sum([v for v in dataset_count.values()])
print('total: ', total)

{
  "B02": 1104,
  "B03": 1646,
  "B05": 1556,
  "B06": 1367,
  "C02": 303,
  "C03": 221,
  "C04": 1109,
  "C05": 4468,
  "D01": 1602,
  "D02": 277,
  "D03": 3502,
  "D04": 1599,
  "D05": 4288,
  "D06": 5309,
  "E05": 112,
  "F06": 81
}
total:  28544


# remove blanks/night time photos

In [8]:
df2_blank = df2.copy().loc[df2['question__species']=='blank']
df2_blank

,capture_id,site,capture_time_local,question__species
162,ENO_S1#B02#1#125,B02,19:29:55,blank
178,ENO_S1#B02#1#137,B02,20:41:29,blank
207,ENO_S1#B02#1#155,B02,19:13:18,blank
229,ENO_S1#B02#1#171,B02,18:08:56,blank
234,ENO_S1#B02#1#176,B02,08:13:21,blank
...,...,...,...,...
13686,ENO_S1#F06#1#39,F06,18:11:41,blank
13694,ENO_S1#F06#1#45,F06,14:16:46,blank
13696,ENO_S1#F06#1#47,F06,11:51:54,blank
13697,ENO_S1#F06#1#48,F06,11:52:57,blank


In [9]:
df2_time = pd.concat([df2.copy().loc[df2['capture_time_local'] < '07:00:00'], \
                 df2.copy().loc[df2['capture_time_local'] > '19:00:00']])
#df2_time = df2_blank.copy() # for CDB: no 'capture_time_local'
df2_time

,capture_id,site,capture_time_local,question__species
91,ENO_S1#B02#1#78,B02,01:52:50,hippopotamus
92,ENO_S1#B02#1#79,B02,02:00:55,hippopotamus
93,ENO_S1#B02#1#80,B02,02:02:54,hippopotamus
94,ENO_S1#B02#1#81,B02,04:24:38,hyenaspotted
131,ENO_S1#B02#1#105,B02,01:40:31,hippopotamus
...,...,...,...,...
13152,ENO_S1#D06#1#1380,D06,22:23:00,blank
13153,ENO_S1#D06#1#1381,D06,22:27:39,blank
13154,ENO_S1#D06#1#1382,D06,22:31:09,blank
13155,ENO_S1#D06#1#1383,D06,22:34:45,blank


In [10]:
df = pd.concat([df2_blank, df2_time])
df = df.join(df1.set_index('capture_id'), on='capture_id', how='inner').drop_duplicates('image_path_rel')
print(len(df['image_path_rel'].unique()))
df

21210


,capture_id,site,capture_time_local,question__species,image_path_rel
162,ENO_S1#B02#1#125,B02,19:29:55,blank,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0341.JPG
178,ENO_S1#B02#1#137,B02,20:41:29,blank,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0361.JPG
207,ENO_S1#B02#1#155,B02,19:13:18,blank,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0409.JPG
229,ENO_S1#B02#1#171,B02,18:08:56,blank,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0447.JPG
229,ENO_S1#B02#1#171,B02,18:08:56,blank,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0448.JPG
...,...,...,...,...,...
11344,ENO_S1#D05#1#1354,D05,19:13:58,duiker,ENO_S1/D05/D05_R1/ENO_S1_D05_R1_IMAG3372.JPG
11345,ENO_S1#D05#1#1355,D05,19:17:08,bushbuck,ENO_S1/D05/D05_R1/ENO_S1_D05_R1_IMAG3373.JPG
11809,ENO_S1#D06#1#16,D06,23:59:44,dikdik,ENO_S1/D06/D06_R1/ENO_S1_D06_R1_IMAG0046.JPG
11833,ENO_S1#D06#1#40,D06,23:39:47,dikdik,ENO_S1/D06/D06_R1/ENO_S1_D06_R1_IMAG0114.JPG


In [11]:
sites = df['site'].unique()

site_count = {}
for s in sites: 
    num = np.int(df.loc[df['site'] == s]['capture_id'].count())
    
    site_count[str(s)] = num #[num, min_date, max_date, min_time, max_time]

print(json.dumps(site_count, indent=2))

total = sum([v for v in site_count.values()])
print(total, ' are blanks or nighttime images')

{
  "B02": 156,
  "B03": 1563,
  "B05": 353,
  "B06": 445,
  "C02": 52,
  "C03": 57,
  "C04": 476,
  "C05": 2789,
  "D01": 1515,
  "D02": 149,
  "D03": 3386,
  "D04": 830,
  "D05": 4197,
  "D06": 5200,
  "E05": 21,
  "F06": 21
}
21210  are blanks or nighttime images


In [12]:
remove_lst = list(df['image_path_rel'])

# choose 60 images

In [13]:
df_remain = df_full.copy().append(df).drop_duplicates(keep=False).reset_index(drop=True)
df_remain

,capture_id,site,capture_time_local,question__species,image_path_rel
0,ENO_S1#B02#1#7,B02,12:02:37,birdother,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0017.JPG
1,ENO_S1#B02#1#7,B02,12:02:37,birdother,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0018.JPG
2,ENO_S1#B02#1#7,B02,12:02:37,birdother,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0019.JPG
3,ENO_S1#B02#1#8,B02,12:39:24,domesticanimal,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0020.JPG
4,ENO_S1#B02#1#8,B02,12:39:24,domesticanimal,ENO_S1/B02/B02_R1/ENO_S1_B02_R1_IMAG0021.JPG
...,...,...,...,...,...
7329,ENO_S1#F06#1#44,F06,14:15:24,lionfemale,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0129.JPG
7330,ENO_S1#F06#1#44,F06,14:15:24,lionfemale,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0130.JPG
7331,ENO_S1#F06#1#46,F06,14:20:09,domesticanimal,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0134.JPG
7332,ENO_S1#F06#1#46,F06,14:20:09,domesticanimal,ENO_S1/F06/F06_R1/ENO_S1_F06_R1_IMAG0135.JPG


In [14]:
full_lst = list(df_full['image_path_rel'])
erase_lst = list(df['image_path_rel'])
remain_lst = list(df_remain['image_path_rel'])
maybe_full_lst = erase_lst + remain_lst

for r in remain_lst:
    assert(r not in df['image_path_rel'])
    
for e in erase_lst:
    assert(e not in df_remain['image_path_rel'])
    
assert(len(maybe_full_lst) == len(full_lst))

In [15]:
full_site_count = {}
for s in sites: 
    num = np.int(df_remain.loc[df_remain['site'] == s]['capture_id'].count())
    content = list(df_remain.loc[df_remain['site'] == s]['image_path_rel']) #['c'] 
    full_site_count[str(s)] = [num, content]

In [16]:
n = 60 # to be saved per site # CDB: 75
rng = np.random.default_rng(2021)
remove_lst2 = []
for k, v in full_site_count.items():
    if v[0]>=n:
        remove_lst2 = remove_lst2 + rng.choice(v[1], size=v[0]-n, replace=False).tolist()
        print(n, 'valid images from', k)
    else:
        remove_lst2 = remove_lst2 + v[1]
        print('not enough valid images for ', k)

60 valid images from B02
60 valid images from B03
60 valid images from B05
60 valid images from B06
60 valid images from C02
60 valid images from C03
60 valid images from C04
60 valid images from C05
60 valid images from D01
60 valid images from D02
60 valid images from D03
60 valid images from D04
60 valid images from D05
60 valid images from D06
60 valid images from E05
60 valid images from F06


# delete unwanted images

In [17]:
remove_lst = remove_lst + remove_lst2
remove_str = ' '.join(remove_lst)
remove_filename = 'remove_' + folder + '.txt' 
with open(remove_filename, 'w') as f:
    f.write(remove_str)
    
f.close()

In [18]:
# replace command w/ proper filename
# !xargs -a remove_ENO.txt rm 

## end